In [2]:
from dotenv import load_dotenv
import os
import openai

# Load environment variables from the .env file
load_dotenv()

# Retrieve the OpenAI API key from environment variables
openai.api_key = os.getenv("OPENAI_API_KEY")

# Check if the API key is loaded
if openai.api_key is None:
    raise ValueError("API key not found. Please check the .env file.")
else:
    print("API key loaded successfully.")

API key loaded successfully.


In [8]:
import os
from openai import OpenAI

# Instantiate the client using environment variable for the API key
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

# Make a chat completion request
chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ]
)

# Print the response
print(chat_completion)

ChatCompletion(id='chatcmpl-AGIX1U8t70m0jHcKkcgo3xzFU7kA1', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='This is a test.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1728447847, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=5, prompt_tokens=12, total_tokens=17, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0)))
This is a test.


In [9]:
print(chat_completion.choices[0].message.content)

This is a test.


In [15]:
def mood_agent(content):
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[{
            "role": "user",
            "content": f"Analyze the mood of this message, focusing on any hypothetical or speculative language that could affect sentiment: {content}"
        }]
    )
    return response.choices[0].message.content

def institutional_investor_agent(content):
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[{
            "role": "user",
            "content": f"Analyze this message as if you are an institutional investor, focusing on long-term impacts on stability and growth potential: {content}"
        }]
    )
    return response.choices[0].message.content


def individual_investor_agent(content):
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[{
            "role": "user",
            "content": f"Analyze this message as if you are an individual investor, focusing on short-term price impact and immediate gains or losses: {content}"
        }]
    )
    return response.choices[0].message.content


def rhetoric_agent(content):
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[{
            "role": "user",
            "content": f"Analyze the rhetorical style of this message, such as sarcasm, exaggeration, or assertive statements, and how these elements affect sentiment: {content}"
        }]
    )
    return response.choices[0].message.content


def dependency_agent(content):
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[{
            "role": "user",
            "content": f"Focus on the speaker’s sentiment in this message, without considering external perspectives or opinions of third parties: {content}"
        }]
    )
    return response.choices[0].message.content


def aspect_agent(content):
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[{
            "role": "user",
            "content": f"Analyze the sentiment toward the main entity (e.g., company or stock ticker) in this message, ignoring unrelated information: {content}"
        }]
    )
    return response.choices[0].message.content


def reference_agent(content):
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[{
            "role": "user",
            "content": f"Identify references to time, price points, or external factors in this message, and analyze how they impact the overall sentiment: {content}"
        }]
    )
    return response.choices[0].message.content


In [16]:
content = "$SBUX this stock has been consolidating and coiling for years. I think many underestimate it. spring how"

# Run each agent on the content
responses = {
    "mood_agent": mood_agent(content),
    "institutional_investor_agent": institutional_investor_agent(content),
    "individual_investor_agent": individual_investor_agent(content),
    "rhetoric_agent": rhetoric_agent(content),
    "dependency_agent": dependency_agent(content),
    "aspect_agent": aspect_agent(content),
    "reference_agent": reference_agent(content)
}

# Display the responses from each agent
for agent, response in responses.items():
    print(f"{agent}: {response}\n")


mood_agent: The mood of the message is cautiously optimistic and anticipatory regarding the stock performance of Starbucks ($SBUX). The use of descriptive verbs like "consolidating" and "coiling" suggests a sense of potential energy and preparation, similar to a spring that is being compressed before it releases. This imagery indicates that the speaker anticipates a significant future movement or change in the stock’s behavior, implying an expectation of upward momentum or a positive breakout.

The phrase "I think many underestimate it" introduces a speculative element, highlighting the speaker's personal belief that others may not fully recognize the stock's potential. This introduces a contrast between the speaker's viewpoint and the general market perception, suggesting that the speaker sees an overlooked value or opportunity.

The final phrase, "spring how" appears to be a typographical or textual truncation. Assuming the intended phrase might be something like “spring now” or “spr

# Mock Tests

In [21]:
# Sample financial message for sentiment analysis
content = "$SBUX this stock has been consolidating and coiling for years. I think many underestimate it. spring how"

In [22]:
# Mock agent responses to simulate natural language sentiment descriptions
def mood_agent(content):
    return "The mood seems cautiously optimistic about future potential."

def institutional_investor_agent(content):
    return "This message suggests a long-term positive outlook due to consolidation."

def individual_investor_agent(content):
    return "Indicates potential for short-term gains but remains uncertain."

def rhetoric_agent(content):
    return "The language implies underestimation by others, hinting at overlooked growth."

def dependency_agent(content):
    return "The speaker seems to hold a positive perspective independently."

def aspect_agent(content):
    return "Focuses specifically on $SBUX, suggesting stability and possible growth."

def reference_agent(content):
    return "No explicit timeframes, but consolidation indicates a buildup for potential movement."


In [23]:
# Collect initial responses from each agent in a dictionary
responses = {
    "mood_agent": mood_agent(content),
    "institutional_investor_agent": institutional_investor_agent(content),
    "individual_investor_agent": individual_investor_agent(content),
    "rhetoric_agent": rhetoric_agent(content),
    "dependency_agent": dependency_agent(content),
    "aspect_agent": aspect_agent(content),
    "reference_agent": reference_agent(content)
}

In [24]:
# Summative Agent with combined logic for proceeding to the next round if inconclusive
def summative_agent(responses, max_rounds=2):
    round_count = 0
    consensus_reached = False
    high_priority_agents = ["institutional_investor_agent", "individual_investor_agent"]
    
    # Store initial responses
    sentiment_summary = {agent: response for agent, response in responses.items()}

    while not consensus_reached and round_count < max_rounds:
        # Step 1: Ask the model to summarize the collective sentiment based on agent responses
        combined_responses = "\n".join([f"{agent}: {response}" for agent, response in sentiment_summary.items()])
        
        # Ask GPT-4 to interpret the overall sentiment, allowing for Positive, Negative, Neutral, or Mixed
        overall_sentiment = client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[{
                "role": "user", 
                "content": f"Based on the following responses from various agents, summarize the overall sentiment as Positive, Negative, Neutral, or Mixed if no clear conclusion can be derived:\n\n{combined_responses}"
            }]
        ).choices[0].message.content.strip().lower()
        
        # Check if a clear consensus is indicated by the response
        if "positive" in overall_sentiment:
            consensus_reached = True
            final_sentiment = "Positive"
        elif "negative" in overall_sentiment:
            consensus_reached = True
            final_sentiment = "Negative"
        elif "neutral" in overall_sentiment:
            consensus_reached = True
            final_sentiment = "Neutral"
        elif "mixed" in overall_sentiment or not consensus_reached:
            # Inconclusive: proceed to the next round
            round_count += 1
            for agent, response in responses.items():
                refined_response = client.chat.completions.create(
                    model="gpt-4-turbo",
                    messages=[
                        {"role": "user", "content": f"Refine your sentiment analysis by reviewing these responses:\n{combined_responses}\nOriginal response: {response}"}
                    ]
                )
                sentiment_summary[agent] = refined_response.choices[0].message.content

    # If no consensus after max rounds, defer to high-priority agents for final decision
    if not consensus_reached:
        high_priority_responses = "\n".join([f"{agent}: {sentiment_summary[agent]}" for agent in high_priority_agents if agent in sentiment_summary])
        final_sentiment = client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[{
                "role": "user", 
                "content": f"Summarize the final sentiment based on high-priority agents alone, as Positive, Negative, Neutral, or Mixed if no clear conclusion can be drawn:\n{high_priority_responses}"
            }]
        ).choices[0].message.content.strip().capitalize()

    # Return the final sentiment and a detailed summary of agent responses
    return {
        "final_sentiment": final_sentiment,
        "summary": sentiment_summary
    }


In [25]:
# Run the summative agent to determine final sentiment
final_output = summative_agent(responses)
print("Final Aggregated Sentiment:", final_output["final_sentiment"])
print("Detailed Summary:", final_output["summary"])

Final Aggregated Sentiment: Positive
Detailed Summary: {'mood_agent': 'The mood seems cautiously optimistic about future potential.', 'institutional_investor_agent': 'This message suggests a long-term positive outlook due to consolidation.', 'individual_investor_agent': 'Indicates potential for short-term gains but remains uncertain.', 'rhetoric_agent': 'The language implies underestimation by others, hinting at overlooked growth.', 'dependency_agent': 'The speaker seems to hold a positive perspective independently.', 'aspect_agent': 'Focuses specifically on $SBUX, suggesting stability and possible growth.', 'reference_agent': 'No explicit timeframes, but consolidation indicates a buildup for potential movement.'}


# Financial Data Source

### Financial News: GDELT

In [5]:
import requests
import pandas as pd
from IPython.display import display

url = "https://api.gdeltproject.org/api/v2/doc/doc"
params = {
    'query': '(economy OR finance)',  # Corrected query with parentheses
    'mode': 'ArtList',
    'format': 'JSON'
}

headers = {
    'User-Agent': 'Mozilla/5.0'
}

# Sending request to GDELT
response = requests.get(url, params=params, headers=headers)

# Verify the response and process the data
if response.status_code == 200:
    if response.text.strip():  # Check if the response is not empty
        try:
            data = response.json()
            articles = pd.DataFrame(data.get('articles', []))
            if 'title' in articles.columns:
                # Displaying only the titles
                display(articles['title'].head())
            else:
                print("No title information found in the response.")
        except ValueError:
            print("Error decoding JSON. Response text:", response.text)
    else:
        print("Received an empty response.")
else:
    print("Error: ", response.status_code, "\nResponse Text: ", response.text)


0    Wapres berikan arahan strategis perkuat ekonom...
1    Wapres ungkap strategi tingkatkan literasi - i...
2    Wapres : Ekonomi Syariah Solusi Inklusif dan B...
3                        报道 ： 中美金融工作组于8月15日至16日在上海举行会议
4                                        财经时评｜金融亦是国之重器
Name: title, dtype: object

### SNS Financial Data: Twitter

In [8]:
import tweepy
import pandas as pd
from IPython.display import display

# Set your keys and tokens (replace placeholders with your actual credentials)
consumer_key = "YOUR_CONSUMER_KEY"
consumer_secret = "YOUR_CONSUMER_SECRET"
access_token = "YOUR_ACCESS_TOKEN"
access_token_secret = "YOUR_ACCESS_TOKEN_SECRET"

# Authenticate with Twitter API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# Fetch tweets with the specified keyword
tweets = []
for tweet in tweepy.Cursor(api.search_tweets, q="economy", lang="en", tweet_mode="extended").items(10):
    tweets.append({'created_at': tweet.created_at, 'user': tweet.user.screen_name, 'text': tweet.full_text})

# Convert to DataFrame
tweets_df = pd.DataFrame(tweets)

# Display the first few tweets
display(tweets_df[['created_at', 'user', 'text']])


Unauthorized: 401 Unauthorized
89 - Invalid or expired token.

### Financial Forum: Reddit API

In [ ]:
import praw
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

reddit = praw.Reddit(
    client_id=os.getenv("CLIENT_ID"),
    client_secret=os.getenv("CLIENT_SECRET"),
    password=os.getenv("PASSWORD"),  # Replace with your Reddit account password
    user_agent=os.getenv("USER_AGENT"),
    username=os.getenv("USERNAME").strip()
)

# Verify Reddit instance is connected
print(f"Connected as: {reddit.user.me()}")


In [ ]:
# Fetch posts from a subreddit as a test
subreddit = reddit.subreddit("financialindependence")
for submission in subreddit.hot(limit=5):
    print(submission.title)
